In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

In [ ]:
device='mps'

In [ ]:
X1=torch.rand(10000,1)
X2=torch.rand(10000,1)
X3=torch.rand(10000,1)
X1.shape, X2.shape, X3.shape

In [ ]:
Y1=((X1+X2+X3)<1).float()
Y2=((1<(X1+X2+X3))&(X1+X2+X3<2)).float()
Y3=((X1+X2+X3)>2).float()
Y1.shape, Y2.shape, Y3.shape

In [ ]:
Data=torch.cat([X1,X2,X3,Y1,Y2,Y3],axis=1)
Data=Data.to(device)
Data.shape

In [ ]:
Data

In [ ]:
train_size=int(len(Data)*0.7)
test_size=len(Data)-train_size
Data=Data[torch.randperm(Data.size(0)) , : ]
train_Data=Data[:train_size, :]
test_Data=Data[train_size:, :]
train_Data.shape, test_Data.shape

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN,self).__init__()
        self.net=nn.Sequential(
            nn.Linear(3,5), nn.ReLU(),
            nn.Linear(5,5), nn.ReLU(),
            nn.Linear(5,5), nn.ReLU(),
            nn.Linear(5,3)
        )
    def forward(self,x):
        y=self.net(x)
        return y

In [ ]:
model=DNN().to(device)
model

In [ ]:
for name,param in model.named_parameters():
    print(name, param.shape, param)

In [ ]:
loss_fn=nn.MSELoss()

In [ ]:
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
epochs=1000
losses=[]

X=train_Data[:,:3]
Y=train_Data[:,3:]

In [ ]:
for epoch in range(epochs):
    Pred=model(X)
    loss=loss_fn(Pred,Y)
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
Fig=plt.figure()
plt.plot(range(epochs),losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
X=test_Data[:,:3]
Y=test_Data[:,3:]

with torch.no_grad():
    Pred=model(X)
    Pred[:,torch.argmax(Pred,axis=1)]=1
    Pred[Pred!=1]=0
    correct=torch.sum((Pred==Y).all(1))
    total=Y.size(0)
    print('Accuracy: ',correct/total)

In [ ]:
torch.save(model,'model.pt')

In [ ]:
new_model=torch.load('model.pt')

In [ ]:
X=test_Data[:,:3]
Y=test_Data[:,3:]

with torch.no_grad():
    Pred=new_model(X)
    Pred[:,torch.argmax(Pred,axis=1)]=1
    Pred[Pred!=1]=0
    correct=torch.sum((Pred==Y).all(1))
    total=Y.size(0)
    print('Accuracy: ',correct/total)